In [14]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import polars as pl
from googleapiclient.discovery import build

In [15]:
# Define el alcance y crea una sesión
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credenciales_api_Google.json', scope)
client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

In [16]:
# Crear una nueva hoja de cálculo
try:
  # Intentar abrir archivo
  sheet = client.open('resumen_utilizaciones_2024')
  print('El archivo ya existe. se abrira para su modificacion')
except:
  # Si no existe crea el archivo
  sheet = client.create('resumen_utilizaciones_2024')
  print('El archivo no existe. se creara uno nuevo')

# ID de la carpeta compartida donde se dejara el archivo
folder_id = '1erowKbJaYVAI8jje4vf8QobjkwqMWm76'

# ID de el archivo creado
id_file = sheet.id
id_file

El archivo ya existe. se abrira para su modificacion


'1kbgcG0vyyGvdLZ1oM3unr1qI5ltx2OyuqwIM1Qfc3Gc'

In [17]:
import cx_Oracle  # o usa cx_Oracle si tu sistema lo requiere
from sqlalchemy import create_engine

In [18]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
# leer variables
host: str = config['oracle']['host']
port: str = config['oracle']['port']
sid: str = config['oracle']['service_name']
user: str = config['oracle']['user']
password: str = config['oracle']['password']

In [19]:
dsn: str = cx_Oracle.makedsn(host, port, sid)
engine = create_engine(f'oracle+cx_oracle://{user}:{password}@{dsn}')

In [20]:
query = """
-- Descripcion: Script para analizar la tabla de utilizaciones por servicio
SELECT
	TO_CHAR(TO_DATE(UT."Fecha de Radicacion", 'YYYY-MM-DD'), 'YYYYMM') AS PERIODO_RADICACION,
	TO_CHAR(TO_DATE(UT."Fecha_prestacion" , 'YYYY-MM-DD'), 'YYYYMM') AS PERIODO_PRESTACION,
	TR."Nombre_Ciudad_DANE" AS MUNICIPIO_PRESTADOR,
	TR."Departamento" AS DEPARTAMENTO_PRESTADOR,
	TR."Regional" AS REGIONAL_PRESTADOR,
	UT."Nombre del prestador",
    UT."Tipo de Cuenta",
    UT."Estado_Factura",
	COUNT(1) AS REGISTROS,
	ROUND(SUM(REPLACE(UT."Valor_Pagado", '.', ','))) AS VALOR_PAGADO,
	ROUND(SUM(REPLACE(UT."VrGlosado Procedimiento", '.', ','))) AS VALOR_GLOSADO,
	ROUND(SUM(REPLACE(UT."VrFacturado Procedimiento", '.', ','))) AS VALOR_FACTURADO
FROM
	TBL_OPE_UTILIZACIONES_2024 UT
    LEFT JOIN "Tb_Regiones" TR
    ON TR."Region" = UT."DANE_Prestador"
GROUP BY
	UT."Nombre del prestador",
	TO_CHAR(TO_DATE("Fecha de Radicacion", 'YYYY-MM-DD'), 'YYYYMM'),
	TO_CHAR(TO_DATE(UT."Fecha_prestacion" , 'YYYY-MM-DD'), 'YYYYMM'),
	TR."Nombre_Ciudad_DANE",
	TR."Departamento",
	TR."Regional",
    UT."Tipo de Cuenta",
    UT."Estado_Factura"
ORDER BY
	VALOR_PAGADO DESC
"""


In [21]:
# Parámetros de lectura en chunks
iter_batches = True
batch_size = 100000  # Número de filas por chunk

# Lista para almacenar los chunks
chunks = []
registros = 0

try:
    # Lee los datos en chunks usando iter_batches
    for df_chunk in pl.read_database(
        query=query,
        connection=engine,
        iter_batches=iter_batches,
        batch_size=batch_size
    ):
        # Agrega cada chunk a la lista
        chunks.append(df_chunk)
        
        registros += df_chunk.shape[0]
        
        print(f"Registros leídos: {registros}")
        
    # Consolida todos los chunks en un solo DataFrame
    df_consolidado = pl.concat(chunks)

except Exception as e:
    print("Error al leer la base de datos:", e)

Registros leídos: 100000
Registros leídos: 194143


In [22]:
df_consolidado.shape

(194143, 12)

In [23]:
df_consolidado.describe()

statistic,PERIODO_RADICACION,PERIODO_PRESTACION,MUNICIPIO_PRESTADOR,DEPARTAMENTO_PRESTADOR,REGIONAL_PRESTADOR,Nombre del prestador,Tipo de Cuenta,Estado_Factura,REGISTROS,VALOR_PAGADO,VALOR_GLOSADO,VALOR_FACTURADO
str,str,str,str,str,str,str,str,str,f64,f64,f64,f64
"""count""","""194143""","""184060""","""194143""","""194143""","""194143""","""194143""","""194143""","""194143""",194143.0,194143.0,182063.0,182063.0
"""null_count""","""0""","""10083""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,0.0,12080.0,12080.0
"""mean""",null,null,null,null,null,null,null,null,431.235388,3.6609e7,5.2837e6,4.9065e7
"""std""",null,null,null,null,null,null,null,null,5146.84935,4.0783e8,3.6623e7,4.7442e8
"""min""","""202401""","""200907""","""ABEJORRAL""","""AMAZONAS""","""1 REG BOGOTA""","""3D RADIOLOGY SAS""","""COMPLEMENTO""","""ENVIADA A TESORERIA""",1.0,-9.7106737e7,0.0,0.0
"""25%""",null,null,null,null,null,null,null,null,3.0,180000.0,0.0,298452.0
"""50%""",null,null,null,null,null,null,null,null,16.0,1.336854e6,0.0,1.950145e6
"""75%""",null,null,null,null,null,null,null,null,102.0,9.28e6,344961.0,1.2956506e7
"""max""","""202412""","""202412""","""ÚTICA""","""VICHADA""","""6 REG CENTRO ORIENTE""","""ZULY YIZETH FORERO GUARIN""","""EVENTO""","""RADICADA""",702463.0,5.5732e10,3.0934e9,6.1175e10


In [24]:
# Mueve el archivo a la carpeta específica
file = drive_service.files().get(fileId=id_file,
                                fields='parents').execute()
previous_parents = ",".join(file.get('parents'))
file = drive_service.files().update(fileId=id_file,
                                    addParents=folder_id,
                                    removeParents=previous_parents,
                                    fields='id, parents').execute()

In [ ]:
import numpy as np
from polars import Decimal

# Convierte el DataFrame a una lista de listas
informacion = df_consolidado.to_numpy().tolist()
# Añade los nombres de las columnas como primera fila
informacion.insert(0, df_consolidado.columns)

# Escribe los datos en la hoja de cálculo
worksheet = sheet.get_worksheet(0)  # selecciona la primera hoja

def safe_check_nan_inf(value):
    if isinstance(value, (int, float, np.integer, np.float64, Decimal)):  # Cambiado np.float a np.float64
        return np.isnan(value) or np.isinf(value)
    return False

# Incluir los encabezados y luego aplicar la verificación de NaN e Inf a los datos
informacion = [[None if safe_check_nan_inf(x) else x for x in fila] for fila in informacion]

worksheet.clear() # Borra el contenido de la hoja

worksheet.update(values=informacion, range_name='A1')  # 'A1' indica la celda inicial donde empezar a escribir

{'spreadsheetId': '1kbgcG0vyyGvdLZ1oM3unr1qI5ltx2OyuqwIM1Qfc3Gc',
 'updatedRange': 'Sheet1!A1:L194144',
 'updatedRows': 194144,
 'updatedColumns': 12,
 'updatedCells': 2295485}

In [26]:
# Obtener la URL de la hoja de cálculo
url = sheet.url
print("URL de la Hoja de Cálculo:", url)
# Compartir la hoja de cálculo con tu cuenta empresarial de Google
email = 'camiloaarias@epssanitas.com'
sheet.share(email, perm_type='user', role='writer')

URL de la Hoja de Cálculo: https://docs.google.com/spreadsheets/d/1kbgcG0vyyGvdLZ1oM3unr1qI5ltx2OyuqwIM1Qfc3Gc


<Response [200]>